<a href="https://colab.research.google.com/github/LinYenShou/114-1/blob/main/HW1_2gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import gradio as gr
import pandas as pd
import gspread
from google.colab import auth
from google.auth import default
import re

# 驗證 Google 帳號
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

sheet_edit_url = "https://docs.google.com/spreadsheets/d/1IPwdD4C5XzEbYe6t9EehoOWguXsGpGq8tbpJlwzjbWE/edit"
gsheets = gc.open_by_url(sheet_edit_url)
sheet = gsheets.sheet1

csv_url = "https://docs.google.com/spreadsheets/d/1IPwdD4C5XzEbYe6t9EehoOWguXsGpGq8tbpJlwzjbWE/export?format=csv&gid=0"
df = pd.read_csv(csv_url)

# --- 定義驗證函式 ---
def validate_date(date):
    if re.match(r"^\d{4}-\d{2}-\d{2}$", date):
        return "✅ 日期格式正確！"
    else:
        return "❌ 日期格式錯誤，請重新輸入 (YYYY-MM-DD)。"

def validate_time(time):
    if re.match(r"^(?:[01]\d|2[0-3]):[0-5]\d$", time):
        return "✅ 時間格式正確！"
    else:
        return "❌ 時間格式錯誤，請重新輸入 (HH:MM)。"

def validate_item(item):
    if item.strip() != "":
        return "✅ 品項已輸入！"
    else:
        return "❌ 品項不能為空。"

def validate_money(money):
    if str(money).isdigit():
        return "✅ 金額格式正確！"
    else:
        return "❌ 金額必須是數字，請重新輸入。"

# --- 修正後的 submit_data 函式 ---
def submit_data(date, time, item, money):
    global df

    if not all([
        re.match(r"^\d{4}-\d{2}-\d{2}$", date),
        re.match(r"^(?:[01]\d|2[0-3]):[0-5]\d$", time),
        str(money).isdigit(),
        item.strip() != ""
    ]):
        return df, "❌ 提交失敗：請檢查所有欄位格式是否正確。"

    try:
        money = int(money)
        new_row_df = pd.DataFrame([{
            "日期": date,
            "時間": time,
            "品項": item,
            "金額": money
        }])
        df = pd.concat([df, new_row_df], ignore_index=True)
        sheet.append_row(new_row_df.values.tolist()[0])
        return df, "✅ 資料新增成功！"
    except Exception as e:
        return df, f"❌ 發生錯誤：{e}"

# --- Gradio 介面設定 ---
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("# 個人記帳小幫手")
    gr.Markdown("請在下方輸入您的支出記錄，系統會自動儲存到 Google Sheet。")
    with gr.Row():
        date_input = gr.Textbox(label="日期", placeholder="YYYY-MM-DD")
        date_status = gr.Text(label="日期狀態")
    with gr.Row():
        time_input = gr.Textbox(label="時間", placeholder="HH:MM")
        time_status = gr.Text(label="時間狀態")
    with gr.Row():
        item_input = gr.Textbox(label="品項")
        item_status = gr.Text(label="品項狀態")
    with gr.Row():
        money_input = gr.Textbox(label="金額", placeholder="請輸入數字")
        money_status = gr.Text(label="金額狀態")
    submit_btn = gr.Button("新增記錄")
    output_df = gr.DataFrame(value=df, label="所有記帳記錄")
    final_status = gr.Text(label="提交狀態", value="")
    date_input.change(fn=validate_date, inputs=date_input, outputs=date_status)
    time_input.change(fn=validate_time, inputs=time_input, outputs=time_status)
    item_input.change(fn=validate_item, inputs=item_input, outputs=item_status)
    money_input.change(fn=validate_money, inputs=money_input, outputs=money_status)
    submit_btn.click(fn=submit_data, inputs=[date_input, time_input, item_input, money_input], outputs=[output_df, final_status])

demo.launch(debug=True)

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://27866b0da6b21aade7.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://27866b0da6b21aade7.gradio.live
